# Smooth Monotonic Networks: Experiments on fully monotonic functions
## General definitions
Among others, we compare against XGBoost, which can be installed via `pip install xgboost`, and 
the Hierarchical Lattice Layer, which can be installed via 
`pip install pmlayer`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import random

import torch 
import torch.nn as nn

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
from tqdm.notebook import tnrange

from xgboost import XGBRegressor

from pmlayer.torch.layers import HLattice

from MonotonicNN import SmoothMonotonicNN, MonotonicNN, MonotonicNNAlt
from MonotonicNNPaperUtils import Progress, total_params, fit_torch

In [ ]:
prefix = "supp-"

## Univariate experiments

In [ ]:
T = 21  # number of trials
ls = 75  # lattice points (k in original paper)
ls_small = 35
K = 6
N_train = 100
N_test = 1000
sigma = 0.01  # noise level

In [ ]:
def generate1D(function_name, sigma=0., random=False, xrange=1., N=50):
    if random:
        x = np.random.rand(N) * xrange
        x = np.sort(x, axis=0)
    else:
        xstep = xrange / N
        x = np.arange(0, xrange, xstep)
    match function_name:
        case 'sigmoid10':
            y = 1. /(1. + np.exp(-(x-xrange/2.) * 10.))
        case 'sq':
            y = x**2
        case 'sqrt':
            y = np.sqrt(x)
    y = y + sigma*np.random.normal(0, 1., N)
    return x.reshape(N, 1), y

In [ ]:
tasks = ['sq', 'sqrt', 'sigmoid10']
methods = ['monotonic_alt', 'smooth', 'xgboost', 'xgboost_val', 'iso', 'lattice']
methods = ['smooth']
N_tasks = len(tasks)
N_methods = len(methods)

MSE_train = np.zeros((N_tasks, N_methods, T))
MSE_test = np.zeros((N_tasks, N_methods, T))
MSE_clip = np.zeros((N_tasks, N_methods, T))
R2_train = np.zeros((N_tasks, N_methods, T))
R2_test = np.zeros((N_tasks, N_methods, T))
X_train = np.zeros((N_tasks, T, N_train))
Y_train = np.zeros((N_tasks, T, N_train))
X_test = np.zeros((N_tasks, T, N_test))
Y_test = np.zeros((N_tasks, T, N_test))
O_test = np.zeros((N_tasks, N_methods, T, N_test))
Active = np.zeros((N_tasks, T))
active = 0

for trial in tnrange(T):
    for task_id, task in enumerate(tasks):
        seed = task_id + trial*N_tasks
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

        x_train, y_train = generate1D(task, sigma=sigma, random=True, N=N_train)
        x_test, y_test   = generate1D(task, sigma=0., random=False, N=N_test)
        X_test[task_id, trial] = x_test.reshape(-1)
        Y_test[task_id, trial] = y_test
        X_train[task_id, trial] = x_train.reshape(-1)
        Y_train[task_id, trial] = y_train
        x_train_torch = torch.from_numpy(x_train.astype(np.float32)).clone()
        y_train_torch = torch.from_numpy(y_train.astype(np.float32)).clone()
        x_test_torch = torch.from_numpy(x_test.astype(np.float32)).clone()
        y_test_torch = torch.from_numpy(y_test.astype(np.float32)).clone()

        for method_id, method in enumerate(methods):
            match method:
                case 'xgboost':             
                    model = XGBRegressor(monotone_constraints=(1,), n_estimators=ls)
                    model.fit(x_train, y_train)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'xgboost_val':             
                    x_train_small, x_val, y_train_small, y_val = train_test_split(x_train, y_train, test_size=.25, random_state=42)
                    model = XGBRegressor(monotone_constraints=(1,), n_estimators=ls, early_stopping_rounds=(ls // 10), verbosity=0)
                    model.fit(x_train_small, y_train_small, eval_set=[(x_train_small, y_train_small), (x_val, y_val)], verbose=0)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'iso':
                    model = IsotonicRegression(y_min=0, y_max=1., out_of_bounds='clip')
                    model.fit(x_train, y_train)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'lattice':
                    model = HLattice(1,torch.tensor([ls], dtype=torch.long),[0])
                    if(trial+task_id==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()
                case 'monotonic':
                    model = MonotonicNN(1, K, K)
                    if(trial+task_id==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch)
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()            
                case 'smooth':
                    model = SmoothMonotonicNN(1, K, K, beta=-1.)
                    if(trial+task_id==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch)
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()
                case 'smooth_sq':
                    model = SmoothMonotonicNN(1, K, K, beta=-1., transform='sq')
                    if(trial+task_id==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch)
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()
                case 'monotonic_alt':
                    model = MonotonicNNAlt(1, K, K)
                    if(trial+task_id==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch)
                    y_pred_train = model(x_train_torch).detach().numpy()
                    model.reset_active_max()
                    y_pred_test = model(x_test_torch).detach().numpy()
                    active, _ = model.active_max()
                case 'xgboost_small':             
                    model = XGBRegressor(monotone_constraints=(1,), n_estimators=ls_small)
                    model.fit(x_train, y_train)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)


            MSE_train[task_id, method_id, trial] = mse(y_train, y_pred_train)
            MSE_test[task_id, method_id, trial] = mse(y_test, y_pred_test)
            MSE_clip[task_id, method_id, trial] = mse(y_test, np.clip(y_pred_test, 0., 1.))
            R2_train[task_id, method_id, trial] = r2(y_train, y_pred_train)
            R2_test[task_id, method_id, trial] = r2(y_test, y_pred_test)
            O_test[task_id, method_id, trial] = y_pred_test.reshape(-1)
            Active[task_id, trial] = active
fn = prefix + "univariate.npz"
np.savez(fn, MSE_train=MSE_train, MSE_test=MSE_test, MSE_clip=MSE_clip, R2_train=R2_train, R2_test=R2_test, 
     X_train=X_train, Y_train=Y_train, X_test=X_test, Y_test=Y_test, O_test=O_test, Active=Active)

In [ ]:
np.set_printoptions(precision=4, floatmode='fixed')
print(methods)
print(np.median(MSE_train, axis=2), '\n')
print(np.median(MSE_test, axis=2), '\n')
print(np.median(MSE_clip, axis=2), '\n\n')

print(Active)
print("max:", Active.max())
print("median:", np.median(Active))
print("count 1:", np.count_nonzero(Active == 1))

## Multivariate experiments

In [ ]:
def generatePoly(dim=2, degree=2, sigma_train=0., sigma_test=0, N_train=100, N_test=100):
    x_train = np.random.rand(N_train, dim)
    x_test = np.random.rand(N_test, dim)
    poly = PolynomialFeatures(degree)  # includes bias
    x_poly_train = poly.fit_transform(x_train)
    x_poly_test = poly.fit_transform(x_test)
    w = np.random.rand(x_poly_train.shape[1])
    w_sum = w.sum()
    y_train = np.sum(x_poly_train * w, axis=1)/w_sum + sigma_train * np.random.normal(0, 1., N_train)
    y_test = np.sum(x_poly_test * w, axis=1)/w_sum + sigma_test * np.random.normal(0, 1., N_test)
    return x_train, y_train, x_test, y_test

In [ ]:
N_train = 500
N_test = 1000
T = 21
trees = 100
trees2 = 200
methods = ['smooth', 'xgboost', 'xgboost_val', 'xgboost2', 'xgboost2_val','lattice', 'lattice_plus']
dims = [2, 4, 6]
degree = 2
ls = [10, 3, 2]
K = 6
sigma = 0.01  # noise level

In [ ]:
N_methods = len(methods)
N_tasks = len(dims)
MSE_train = np.zeros((N_tasks, N_methods, T))
MSE_test = np.zeros((N_tasks, N_methods, T))
MSE_clip = np.zeros((N_tasks, N_methods, T))
R2_train = np.zeros((N_tasks, N_methods, T))
R2_test = np.zeros((N_tasks, N_methods, T))
O_test = np.zeros((N_tasks, N_methods, T, N_test))
for task_id, dim in enumerate(dims):
    for trial in tnrange(T):
        seed = task_id + trial*N_tasks
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

        lattice_sizes = list(np.ones(dim)*ls[task_id])
        lattice_sizes_plus = list(np.ones(dim)*(ls[task_id] + 1))
        lattice_sizes_tensor = torch.tensor(lattice_sizes, dtype=torch.long)
        lattice_sizes_tensor_plus = torch.tensor(lattice_sizes_plus, dtype=torch.long)
        increasing = [1] * dim

        x_train, y_train, x_test, y_test = generatePoly(dim, degree=degree, sigma_train=sigma, sigma_test=0., N_train=N_train, N_test=N_test)
        x_train_torch = torch.from_numpy(x_train.astype(np.float32)).clone()
        y_train_torch = torch.from_numpy(y_train.astype(np.float32)).clone()
        x_test_torch = torch.from_numpy(x_test.astype(np.float32)).clone()
        y_test_torch = torch.from_numpy(y_test.astype(np.float32)).clone()

        for method_id, method in enumerate(methods):
            match method:
                case 'xgboost':             
                    model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees)
                    model.fit(x_train, y_train)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'xgboost_val':             
                    x_train_small, x_val, y_train_small, y_val = train_test_split(x_train, y_train, test_size=.25, random_state=42)
                    model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees, 
                                         early_stopping_rounds=(trees // 10), verbosity=0)
                    model.fit(x_train_small, y_train_small, eval_set=[(x_train_small, y_train_small), (x_val, y_val)], verbose=0)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'xgboost2':             
                    model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees2)
                    model.fit(x_train, y_train)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'xgboost2_val':             
                    x_train_small, x_val, y_train_small, y_val = train_test_split(x_train, y_train, test_size=.25, random_state=42)
                    model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees2, 
                                         early_stopping_rounds=(trees2 // 10), verbosity=0)
                    model.fit(x_train_small, y_train_small, eval_set=[(x_train_small, y_train_small), (x_val, y_val)], verbose=0)
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)
                case 'lattice':
                    model = HLattice(dim, lattice_sizes_tensor, increasing)
                    if(trial==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()      
                case 'lattice_plus':
                    model = HLattice(dim, lattice_sizes_tensor_plus, increasing)
                    if(trial==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()      
                case 'smooth':
                    model = SmoothMonotonicNN(dim, K, K, beta=-1.)
                    if(trial==0):
                        print(method, total_params(model), "parameters")
                    fit_torch(model, x_train_torch, y_train_torch)
                    y_pred_train = model(x_train_torch).detach().numpy()
                    y_pred_test = model(x_test_torch).detach().numpy()

            MSE_train[task_id, method_id, trial] = mse(y_train, y_pred_train)
            MSE_test[task_id, method_id, trial] = mse(y_test, y_pred_test)
            MSE_clip[task_id, method_id, trial] = mse(y_test, np.clip(y_pred_test, 0., 1.))
            R2_train[task_id, method_id, trial] = r2(y_train, y_pred_train)
            R2_test[task_id, method_id, trial] = r2(y_test, y_pred_test)
            O_test[task_id, method_id, trial] = y_pred_test.reshape(-1)
fn = prefix + "multivariate.npz"
np.savez(fn, MSE_train=MSE_train, MSE_test=MSE_test, MSE_clip=MSE_clip, R2_train=R2_train, R2_test=R2_test, O_test=O_test)